## 작사가 인공지능 만들기 🎼

___

💬 인공지능이 인간의 언어를 이해할 수 있을까요? 더 나아가 스스로 글을 쓸 수도 있을까요? 오늘의 프로젝트는 **수많은 노래 가사들을 학습하고 스스로 가사를 쓰는 작사가 인공지능을 만드는 것입니다!**

인간이 사용하는 언어를 이해하고 활용하는 머신러닝 분야를 **자연어처리(NLP:Natural Language Processing)** 라고 합니다. 음성인식 기술이나 파파고의 번역기술, 작문 AI GPT-3 등도 이 분야에 속합니다. 오늘은 NLP에서 자주 쓰이는 RNN 네트워크를 이용하여 모델을 구성할 것입니다.

___

### 0. 데이터 전처리

순환신경망(RNN)은 이전에 입력받은 토큰을 기준으로 (통계적으로) 다음 토큰을 생성합니다. RNN 네트워크를 학습시키기 위해 **새로은 문장을 생성하는 토큰인`<start>`와 문장이 끝났음을 알리는 `<end>` 토큰을 이용해서 데이터를 생성한 후, 소스 데이터와 타겟 데이터로 분리할 것입니다.**

훌륭한 작사가가 되려면 이미 쓰인 좋은 가사들을 많이 알면 도움이 되겠죠?
아델부터 리한나까지 유명 가수들의 노래가사들을 모은 데이터를 활용해 모델을 학습시킬 것입니다. 

먼저 가사데이터를 불러오고 `raw_corpus`라는 리스트에 한 줄씩 담아줍니다.

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


첫 세 줄을 불러와봤습니다. 구글링해보니 캐나다 출신의 가수 Leonard Cohen의 Hallelujah라는 곡이네요.

* **토큰화(Tokenize)**

**토큰화는 자연어 문장을 모델이 처리하기 쉬운 크기로 끊어 단위화하는 것을 의미합니다.** 영어로 생각하면 단어 단위로, 한국어로 생각하면 어절을 단위로 토큰화를 해야겠네요. (더 정교하게 모델링하려면 형태소 단위로 끊어야 하려나요)

여기서 다루는 데이터는 영어 가사이기 때문에 띄어쓰기를 단위로 끊어주면 됩니다. 다만 대문자 소문자의 구분이나 불필요한 문장 부호 등은 없애는 게 좋겠네요. 그리고 후에 소스와 타켓데이터로 분리하기 위해 문장 앞 뒤에 `<start>`와 `<end>`를 붙여줘야합니다. **원하는 방식으로 문장을 정제해 줄 함수 `preprocess_sentence`를 정의합니다.**

In [2]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

`preprocess_sentence` 함수를 이용하여 원하는 형식으로 정제한 문장을 `corpus` 리스트에 담아줍니다. 너무 긴 문장은 다른 데이터들이 과도한 padding을 갖게 하고, 작사가 모델을 학습시키기에 알맞지 않을 수 있습니다. **따라서 토큰의 개수가 15개를 넘기는 경우 데이터에서 제외시키겠습니다.**

In [3]:
corpus = []

for sentence in raw_corpus:
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    # <start>와 <end> 토큰을 제외하고 토큰의 개수가 15개가 넘는 경우 제외합니다
    if len(preprocessed_sentence.split()) > 17: continue
    
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

문장이 잘 정제되어 담겨있음을 확인할 수 있습니다.

문장을 띄어쓰기 단위로 끊어 읽을 수 있으니 토큰화가 다 된 것 같지만, 컴퓨터의 입장에서는 아닙니다. 컴퓨터는 언어를 이해하는 것이 아니라 언어를 숫자로 변형한 데이터를 이해하니까요. 따라서 **우리가 만든 문장들을 숫자로 변환해줘야 합니다. 이 과정을 벡터화(vectorize)라고 하고, 숫자로 변환된 데이터를 텐서(tensor)라고 합니다.**

`corpus`에 담아둔 가사들을 텐서플로우의 `Tokenizer`와 `pad_sequences`를 사용하여 컴퓨터가 알아들을 수 있게 토큰화합니다.

In [4]:
import tensorflow as tf

def tokenize(corpus):
    # 15000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 15000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2930 ...    0    0    0]
 [   2   32    7 ...    0    0    0]
 ...
 [   2  261  192 ...    0    0    0]
 [   2  132    4 ...   10 1070    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4659e0ed30>


우리가 가지고 있던 가사들이 숫자 데이터(tensor)로 바뀌어서 저장된 것을 볼 수 있습니다.

그럼 각 숫자 인덱스에 배당된 단어를 살펴볼까요. `tokenizer`라는 곳에 단어사전이 생성되어 있습니다.

In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


이제 텐서들을 `<start>`로 시작하는 소스 데이터와 `<end>`로 끝나는 타겟 데이터로 분리해 줍니다. 

In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  308   62   55    9  972 6004    3    0    0    0
    0    0]
[  50    4   95  308   62   55    9  972 6004    3    0    0    0    0
    0    0]


여태까지 토큰화한 데이터를 `tensorflow`에서 활용될 데이터셋 형태로 만들어 줍니다. `tf.data.Dataset.from_tensor_slices()`를 이용하여 `tf.data.Dataset` 객체를 생성합니다.

In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 16), (256, 16)), types: (tf.int32, tf.int32)>

이제 훈련 데이터와 평가 데이터를 분리합니다. `sklearn`의 `train_test_split()`을 이용합니다. 학습 데이터와 평가 데이터의 비율은 8:2 입니다.

In [8]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=2)
                                                        

print('enc_train 개수: ', len(enc_train),', enc_val 개수: ', len(enc_val))

enc_train 개수:  139852 , enc_val 개수:  34964


In [9]:
enc_train.shape, dec_train.shape

((139852, 16), (139852, 16))

___

### 1. 모델 설계하기

1개의 `Embedding` 레이어, 2개의 `LSTM` 레이어, 1개의 `Dense` 레이어로 구성된 모델을 생성합니다. 

`Embedding` 레이어는 텐서의 숫자를 워드 벡터로 바꿔줍니다. 워드 벡터란 해당 단어의 의미를 해부한 벡터입니다. 즉 `Embedding` 레이어를 통해서 단어의 의미와 단어들 사이의 연관성을 표현합니다.

`LSTM`은 `Long Short-Term Memory`의 약자로(세상 이런 모순적인 말이 있나?) RNN 네트워크의 한 종류입니다.. RNN의 특징은 시퀀스한 데이터에 기반하여 타겟 데이터를 예측하는데, 필요한 정보까지의 거리가 먼 경우 어려움이 생깁니다. 이런 문제를 "긴 기간의 의존성(long-term dependencies)"라고 하는데, `LSTM`은 이를 해결하기 위해 만들어진 새로운 RNN 네트워크입니다.

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

`embedding_size`: 워드 벡터의 차원 수(단어가 추상적으로 표현되는 정도)

`hidden_size`: LSTM 레이어의 hidden state 의 차원수(≒일하는 일꾼의 수!)

👉 loss의 최소화를 위해서 수치 조정

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 16, 15001), dtype=float32, numpy=
array([[[ 2.1813596e-05, -1.3488010e-04,  1.1096840e-04, ...,
          1.2949065e-04,  3.3164534e-04,  1.7096275e-04],
        [ 5.9822265e-05, -1.0874363e-04,  5.5842876e-05, ...,
          2.3249318e-05,  6.4104836e-04,  3.9525999e-04],
        [ 2.3417514e-04, -4.2893909e-05, -1.6348301e-04, ...,
          2.7113850e-04,  6.5880781e-04,  3.5857587e-04],
        ...,
        [ 2.6840535e-03,  2.0165492e-03,  1.1301689e-03, ...,
          3.0082683e-03, -2.3508999e-03, -3.0586624e-03],
        [ 3.0192370e-03,  2.2273872e-03,  1.3146029e-03, ...,
          3.1658052e-03, -2.5453919e-03, -3.3218863e-03],
        [ 3.3144697e-03,  2.4131318e-03,  1.4891539e-03, ...,
          3.2713385e-03, -2.7116395e-03, -3.5514084e-03]],

       [[ 2.1813596e-05, -1.3488010e-04,  1.1096840e-04, ...,
          1.2949065e-04,  3.3164534e-04,  1.7096275e-04],
        [ 2.7969852e-05, -4.3367987e-04,  4.6673900e-04, ...,
          1.3746547e-04, 

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


___

### 2. 모델 학습시키기

모델을 학습시키고 `loss`값이 2.2 수준이 될 때까지 `embedding_size`와 `hidden_size`를 조절합니다.

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
682/682 [==============================] - 135s 194ms/step - loss: 2.9956
Epoch 2/10
682/682 [==============================] - 140s 205ms/step - loss: 2.5568
Epoch 3/10
682/682 [==============================] - 140s 206ms/step - loss: 2.4081
Epoch 4/10
682/682 [==============================] - 141s 206ms/step - loss: 2.2949
Epoch 5/10
682/682 [==============================] - 141s 206ms/step - loss: 2.1971
Epoch 6/10
682/682 [==============================] - 141s 206ms/step - loss: 2.1080
Epoch 7/10
682/682 [==============================] - 141s 206ms/step - loss: 2.0263
Epoch 8/10
682/682 [==============================] - 141s 206ms/step - loss: 1.9510
Epoch 9/10
682/682 [==============================] - 140s 206ms/step - loss: 1.8815
Epoch 10/10
682/682 [==============================] - 140s 206ms/step - loss: 1.8162


😏 휴우~ 시간이 굉장히 오래걸리네요! 

변수를 따로 조절하지 않았는데도 loss 값이 2.2보다 적게 나왔습니다!

___

### 3. 잘 만들어졌는지 평가하기

작사가 모델을 이용해 노랫말을 생성해 봅시다!

In [14]:
def generate_text(model, tokenizer, init_sentence="I", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [82]:
generate_text(model, tokenizer, init_sentence="<start> love", max_len=20)

'<start> love is a beautiful thing <end> '

오~ 그럴 듯한 가사를 써 내는군요 🤩

In [16]:
generate_text(model, tokenizer, init_sentence="<start> hey", max_len=20)

'<start> hey , hey , hey , hey , hey <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> baby", max_len=20)

'<start> baby , baby , baby , baby , baby , baby , baby <end> '

'Hallelujah'의 예처럼, 반복되는 가사가 학습데이터로 많이 들어가서 그런지 같은 단어가 반복되는 가사를 많이 만들어 내네요.

In [140]:
generate_text(model, tokenizer, init_sentence="<start> come", max_len=20)

'<start> come on motherfuckers come on <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> what", max_len=20)

'<start> what you want nixga what you what you want nixga <end> '

힙합 가사의 영향인지 욕도 많이.. 나오네요.. 😇

In [20]:
generate_text(model, tokenizer, init_sentence="<start> let", max_len=20)

'<start> let s go <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> why", max_len=20)

'<start> why you wanna see me in the dark <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> I", max_len=20)

'<start> i m the one <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you re the only one who knows <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s a sportsman , a gypsy , a gypsy <end> '

다양한 단어를 시작 토큰으로 넣어줘도 꽤 괜찮은 가사를 만들어내는 것을 볼 수 있습니다!

___

하이퍼파라미터의 변화에 따른 차이를 보기 위해서 `model2`도 만들어 봅니다.

`embedding_size`와 `hidden_size`를 각각 두 배 씩 늘려줍니다.

In [26]:
embedding_size = 512
hidden_size = 2048
model2 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [27]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model2.compile(loss=loss, optimizer=optimizer)
model2.fit(dataset, epochs=10)

Epoch 1/10
682/682 [==============================] - 402s 562ms/step - loss: 2.9963
Epoch 2/10
682/682 [==============================] - 385s 565ms/step - loss: 2.5025
Epoch 3/10
682/682 [==============================] - 387s 567ms/step - loss: 2.2813
Epoch 4/10
682/682 [==============================] - 387s 567ms/step - loss: 2.0832
Epoch 5/10
682/682 [==============================] - 387s 567ms/step - loss: 1.8971
Epoch 6/10
682/682 [==============================] - 387s 567ms/step - loss: 1.7232
Epoch 7/10
682/682 [==============================] - 387s 567ms/step - loss: 1.5579
Epoch 8/10
682/682 [==============================] - 387s 567ms/step - loss: 1.4015
Epoch 9/10
682/682 [==============================] - 387s 567ms/step - loss: 1.2577
Epoch 10/10
682/682 [==============================] - 387s 567ms/step - loss: 1.1314


시간이 전보다도 더 걸렸네요.

In [28]:
generate_text(model2, tokenizer, init_sentence="<start> I", max_len=20)

'<start> i m gonna make you feel so good <end> '

In [29]:
generate_text(model2, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you know i m bad , i m bad you know it <end> '

In [30]:
generate_text(model2, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s got me runnin round and round she tastes like the sunshine kissing me <end> '

In [31]:
generate_text(model2, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s the only one for me <end> '

In [32]:
generate_text(model2, tokenizer, init_sentence="<start> baby", max_len=20)

'<start> baby , i got a plan <end> '

In [33]:
generate_text(model2, tokenizer, init_sentence="<start> hey", max_len=20)

'<start> hey , hey , hey , hey , hey , hey <end> '

In [34]:
generate_text(model2, tokenizer, init_sentence="<start> tell me", max_len=20)

'<start> tell me what i wanna hear <end> '

여전히 가사를 잘 뽑아냅니다. 반복되는 가사는 이전 모델보다 조금 줄었네요.

`model3`도 만들어 봅니다.

`embedding_size`와 `hidden_size`를 각각 두 배 씩 줄여줍니다.

In [35]:
embedding_size = 128
hidden_size = 512
model3 = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [36]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model3.compile(loss=loss, optimizer=optimizer)
model3.fit(dataset, epochs=10)

Epoch 1/10
682/682 [==============================] - 63s 88ms/step - loss: 3.2883
Epoch 2/10
682/682 [==============================] - 61s 89ms/step - loss: 2.7303
Epoch 3/10
682/682 [==============================] - 61s 89ms/step - loss: 2.5833
Epoch 4/10
682/682 [==============================] - 61s 90ms/step - loss: 2.4891
Epoch 5/10
682/682 [==============================] - 61s 90ms/step - loss: 2.4107
Epoch 6/10
682/682 [==============================] - 61s 90ms/step - loss: 2.3384
Epoch 7/10
682/682 [==============================] - 61s 90ms/step - loss: 2.2699
Epoch 8/10
682/682 [==============================] - 61s 90ms/step - loss: 2.2043
Epoch 9/10
682/682 [==============================] - 61s 90ms/step - loss: 2.1410
Epoch 10/10
682/682 [==============================] - 61s 90ms/step - loss: 2.0808


In [37]:
generate_text(model3, tokenizer, init_sentence="<start> I", max_len=20)

'<start> i m the one that s gon be alright <end> '

In [38]:
generate_text(model3, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you re the only one that s in the zone <end> '

In [39]:
generate_text(model3, tokenizer, init_sentence="<start> she", max_len=20)

'<start> she s a <unk> , <end> '

In [40]:
generate_text(model3, tokenizer, init_sentence="<start> he", max_len=20)

'<start> he s a <unk> , <end> '

In [41]:
generate_text(model3, tokenizer, init_sentence="<start> baby", max_len=20)

'<start> baby , baby , baby , baby , baby <end> '

In [42]:
generate_text(model3, tokenizer, init_sentence="<start> hey", max_len=20)

'<start> hey , hey , hey , hey , hey , hey , hey , hey , hey , hey '

In [43]:
generate_text(model3, tokenizer, init_sentence="<start> tell me", max_len=20)

'<start> tell me what you want <end> '

In [44]:
generate_text(model3, tokenizer, init_sentence="<start> what", max_len=20)

'<start> what you want nixga what you want nixga what you want nixga <end> '

전체적으로 `model3`는 첫번째 `model`과 비슷한 느낌이네요.

___

### 🤔 회고

#### 1. 결과 이미지를 바로바로 확인하면서 진행했던 CV 프로젝트와는 다르게 데이터 전처리하는 과정이 굉장히 길고 복잡하게 느껴져서 초반부에는 시간이 오래걸렸습니다.

#### 2. 그런데 막상 모델을 만들고 직접 가사를 생성해보니 이렇게 재미있을 수가 있을까요? NLP... 넘나 매력적인 분야인 것 같습니다.

#### 3. 익플 프로젝트는 말그대로 '탐험'이기 때문에 모델을 깊게 연구하지는 않고 바로바로 활용하지만 단어 사이의 의미 벡터를 만들고 한 단어 다음에 나올 단어를 통계적으로 추출해낸다는 것이 놀라운 기술이었습니다. 

#### 4. **`Embedding layer`와 `RNN layer`가 어떤 원리로 작용하는지 좀 더 깊이 알아봐야겠다는 생각이 들었습니다.**

#### 5. 아 그런데, `train` 데이터와 `test` 데이터는 왜 굳이 나뉜거죠..? `train` 데이터로만 모델을 만들고 `test`로 실제 예측값과 비교를 해봤어야 하는 건가요..? 😵

#### 6. 하이퍼파라미터 값을 바꾸면서 모델을 만들어봤는데 개인적으로는 첫번째 `model2`가 가장 자연스럽게 느껴졌습니다. 수치적인 결과값이 나오는게 아니어서 모델의 설계자에 따라서 최종 모델이 다르게 설정될 것 같네요.